In [18]:
import OpenDartReader
import pandas as pd
import numpy as np
import time
from tqdm import tqdm

In [12]:
my_api = open("dart_api.txt").read()[1:]
dart = OpenDartReader(my_api)

In [13]:
# Symbol 컬럼을 숫자로 인식하면 00123이 123으로 변환될 가능성이 있으므로 자료형을 str로 설정

stock_list = pd.read_csv(
    "data/종목정보.txt",
    encoding="euc-kr",
    sep="\t",
    usecols=["Name", "Symbol"],
    dtype=str
)

In [14]:
report = dart.finstate("005380", 2020)

# fs_nm 컬럼이 연결재무제표인 행이 있으면 이 행을 필터링. 없으면 재무제표인 행을 사용. 연결재무제표를 발표하는 기업은 연결재무제표 행이 있음
# account_nm 컬럼에서 자산총계, 부채총계, 자본총계, 매출액, 영업이익, 당기순이익인 행을 필터링해서 thstrm_amount, frmtrm_amount, bfefrmtrm_amount 컬럼의 값을 가져옴

In [15]:
# 데이터 파싱

def find_fins_ind_list(stock_code, stock_name, year, ind_list):
    try:
        report = None
        report = dart.finstate(stock_code, year)
    except:
        pass

    if report is None:
        # 리포트가 없으면 당기, 전기, 전전기 값 모두 제거
        data =[[stock_name, year] + [np.nan] * len(ind_list)]
        data.append([stock_name, year - 1] + [np.nan] * len(ind_list))
        data.append([stock_name, year - 1] + [np.nan] * len(ind_list))

    else:
        report = report[report["account_nm"].isin(ind_list)]  # account_nm으로 필터링

        # 연결재무제표가 있으면 그대로 쓰고 없으면 재무제표 사용
        if sum(report["fs_nm"] == "연결재무제표") > 0:
            report = report.loc[report["fs_nm"] == "연결재무제표"]
        else:
            report = report.loc[report["fs_nm"] == "재무제표"]

        data = []
        for y, c in zip([year, year-1, year-2], ["thstrm_amount", "frmtrm_amount", "bfefrmtrm_amount"]):
            record = [stock_name, y]
            for ind in ind_list:
                if sum(report["account_nm"] == ind) > 0:
                    value = report.loc[report["account_nm"] == ind, c].iloc[0]
                else:
                    value = np.nan
                record.append(value)
            data.append(record)

    return pd.DataFrame(data, columns=["기업", "연도"]+ind_list)

In [16]:
ind_list = ["자산총계", "부채총계", "자본총계", "매출액", "영업이익", "당기순이익"]
find_fins_ind_list("005930", "삼성전자", 2020, ind_list)

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익
0,삼성전자,2020,"378,235,718,000,000","102,287,702,000,000","275,948,016,000,000","236,806,988,000,000","35,993,876,000,000","26,407,832,000,000"
1,삼성전자,2019,"352,564,497,000,000","89,684,076,000,000","262,880,421,000,000","230,400,881,000,000","27,768,509,000,000","21,738,865,000,000"
2,삼성전자,2018,"339,357,244,000,000","91,604,067,000,000","247,753,177,000,000","243,771,415,000,000","58,886,669,000,000","44,344,857,000,000"


In [25]:
# 데이터 수집
# 모든 기업과 연도에 대한 주요 재무지표 가져오기

data = pd.DataFrame()

for code, name in tqdm(stock_list[["Symbol", "Name"]].values):
    for year in [2015, 2018, 2020]:
        result = find_fins_ind_list(code, name, year, ind_list)
        data = pd.concat([data, result], axis=0, ignore_index=True)
        time.sleep(0.1)

100%|██████████| 2370/2370 [1:30:37<00:00,  2.29s/it]  


In [26]:
# 데이터 정제
# 세 년도에서 전전기, 전기, 당기 데이터를 가져왔으므로 2018년이 중복됨
# 2018년 데이터를 드롭하고 기업, 연도 컬럼을 기준으로 정렬

data.drop_duplicates(inplace=True)
data.sort_values(by=["기업", "연도"], inplace=True)

In [27]:
data.head()

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익
2,3S,2013,"67,660,583,293","19,883,875,822","47,776,707,471","32,236,446,153","1,859,291,010","701,146,938"
1,3S,2014,"71,312,012,159","28,319,186,484","42,992,825,675","28,862,985,297","-2,540,739,501","-4,760,633,476"
0,3S,2015,"67,359,971,696","29,420,163,240","37,939,808,456","23,210,541,648","-4,157,989,321","-5,599,032,927"
5,3S,2016,"57,746,389,196","23,142,629,825","34,603,759,371","23,742,421,655","1,176,535,923","-3,403,437,816"
4,3S,2017,"53,706,903,697","24,527,236,916","29,179,666,781","26,486,287,918","-2,421,224,869","-8,803,810,083"


In [28]:
# ,와 -을 제거

def convert_str_to_float(value):
    if type(value) == float:
        return value
    elif value == "-":  # 음수에도 하이픈이 있기 때문에 replace를 쓰면 안됨
        return 0
    else:
        return float(value.replace(",", ""))

for ind in ind_list:
    data[ind] = data[ind].apply(convert_str_to_float)

In [29]:
data.head()

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익
2,3S,2013,6.766058e+10,1.988388e+10,4.777671e+10,3.223645e+10,1.859291e+09,7.011469e+08
1,3S,2014,7.131201e+10,2.831919e+10,4.299283e+10,2.886299e+10,-2.540740e+09,-4.760633e+09
0,3S,2015,6.735997e+10,2.942016e+10,3.793981e+10,2.321054e+10,-4.157989e+09,-5.599033e+09
5,3S,2016,5.774639e+10,2.314263e+10,3.460376e+10,2.374242e+10,1.176536e+09,-3.403438e+09
4,3S,2017,5.370690e+10,2.452724e+10,2.917967e+10,2.648629e+10,-2.421225e+09,-8.803810e+09


In [30]:
# 부채비율 계산
# 부채비율 = 자본총계 / 자산총계 * 100

data["부채비율"] = data["자본총계"] / data["자산총계"] * 100
data["부채비율"].head()

2    70.612320
1    60.288336
0    56.323967
5    59.923676
4    54.331315
Name: 부채비율, dtype: float64

In [31]:
# 매출액 증가율, 영업이익 증가율, 당기순이익 증가율 계싼
# 매출액 증가율 = (당기 매출액 - 전기 매출액) / 전기 매출액 * 100
# 영업이익 증가율 = (당기 영업이익 - 전기 영업이익) / 전기 영업이익 * 100
# 당기순이익 증가율 = (당기 당기순이익 - 전기 당기순이익) / 전기 당기순이익 * 100

# 기업, 연도 컬럼을 기준으로 오름차순 정렬되어있으므로 간단하게 코드 작성 가능

data["매출액증가율"] = data["매출액"].diff() / data["매출액"] * 100
data.loc[data["연도"] == 2013, "매출액증가율"] = np.nan  # data 시작이 2013년도이기 때문에 이 해의 이전년도 데이터는 다른 종목의 데이터. 따라서 2013년도 데이터는 없애야함
data["영업이익증가율"] = data["영업이익"].diff() / data["영업이익"] * 100
data.loc[data["연도"] == 2013, "영업이익증가율"] = np.nan
data["당기순이익증가율"] = data["당기순이익"].diff() / data["당기순이익"] * 100
data.loc[data["연도"] == 2013, "당기순이익증가율"] = np.nan

In [32]:
# 위 지표들 중 음수가 들어가있는 값이 있으면 증가율이 이상하게 나옴. 보조지표로 이를 보완

def add_state(data, col):
    data[col + "_상태"] = np.nan
    value = data[col].values
    cur_value = value[1:]
    pre_value = value[:-1]

    # 흑자지속
    cond1 = (cur_value > 0) & (pre_value > 0)
    cond1 = np.insert(cond1, 0, np.nan)

    # 적자지속
    cond2 = (cur_value <= 0) & (pre_value <= 0)
    cond2 = np.insert(cond2, 0, np.nan)

    # 흑자전환
    cond3 = (cur_value > 0) & (pre_value <= 0)
    cond3 = np.insert(cond3, 0, np.nan)

    # 적자전환
    cond4 = (cur_value <= 0) & (pre_value > 0)
    cond4 = np.insert(cond4, 0, np.nan)

    data.loc[cond1, col+"_상태"] = "훅자지속"
    data.loc[cond2, col+"_상태"] = "적자지속"
    data.loc[cond3, col+"_상태"] = "훅자전환"
    data.loc[cond4, col+"_상태"] = "적자전환"

In [33]:
add_state(data, "매출액")
add_state(data, "영업이익")
add_state(data, "당기순이익")

In [34]:
# ROA
# ROA = 당기순이익 / 자산총계 * 100

data["ROA"] = data["당기순이익"] / data["자산총계"] * 100

In [35]:
# ROE
# ROE = 당기순이익 / 평균 자기자본 * 100
# 평균 자기자본 = (전기 자본총계 + 당기 자본총계) / 2

average_equity = data["자본총계"].rolling(2).mean()
data["ROE"] = data["당기순이익"] / average_equity * 100
data.loc[data["연도"] == 2013, "ROE"] = np.nan

In [36]:
data.to_csv("data/주요재무지표.csv", index=False, encoding="euc-kr")

In [37]:
data

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,부채비율,매출액증가율,영업이익증가율,당기순이익증가율,매출액_상태,영업이익_상태,당기순이익_상태,ROA,ROE
2,3S,2013,6.766058e+10,1.988388e+10,4.777671e+10,3.223645e+10,1.859291e+09,7.011469e+08,70.612320,NaN,NaN,NaN,적자전환,적자전환,적자전환,1.036271,NaN
1,3S,2014,7.131201e+10,2.831919e+10,4.299283e+10,2.886299e+10,-2.540740e+09,-4.760633e+09,60.288336,-11.687845,173.179128,114.728018,훅자지속,적자전환,적자전환,-6.675781,-10.489496
0,3S,2015,6.735997e+10,2.942016e+10,3.793981e+10,2.321054e+10,-4.157989e+09,-5.599033e+09,56.323967,-24.352916,38.894997,14.974005,훅자지속,적자지속,적자지속,-8.312107,-13.836280
5,3S,2016,5.774639e+10,2.314263e+10,3.460376e+10,2.374242e+10,1.176536e+09,-3.403438e+09,59.923676,2.240210,453.409466,-64.511098,훅자지속,훅자전환,적자지속,-5.893767,-9.383155
4,3S,2017,5.370690e+10,2.452724e+10,2.917967e+10,2.648629e+10,-2.421225e+09,-8.803810e+09,54.331315,10.359573,148.592592,61.341308,훅자지속,적자전환,적자지속,-16.392325,-27.605322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21321,힘스,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21325,힘스,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21324,힘스,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21328,힘스,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
